In [1]:
!pip install groq  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.4 MB/s eta 0:00:00


In [2]:
pip install groq pandas scikit-learn sentence-transformers numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3.9.90:
      Successfully uninstalled nvidia-curand-cu12-10.3.9.90
  Attempting

In [3]:
from groq import Groq
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np
import os

2025-05-25 15:08:20.684547: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748185700.877161      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748185700.934492      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Set your API key (store this securely)
os.environ["GROQ_API_KEY"] = "your api key here"

client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [ ]:
class MedicalRAGSystem:
    def __init__(self):
        self.embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

        self.client = Groq(api_key='your api key here')
        self.load_data()
    
    def load_data(self):
        """Load and embed the cancer Q&A dataset"""
        df = pd.read_csv('/kaggle/input/cancer-q-and-a-dataset/CancerQA.csv')
        cancer_df = df[df['topic'] == 'cancer'].dropna()
        
        self.questions = cancer_df['Question'].tolist()
        self.answers = cancer_df['Answer'].tolist()
        
        self.question_embeddings = self.embedding_model.encode(self.questions)
    
    def retrieve_most_similar(self, query: str, threshold: float = 0.5) -> dict:
        """Retrieve most similar question and answer"""
        query_embedding = self.embedding_model.encode([query])
        similarities = cosine_similarity(query_embedding, self.question_embeddings)[0]
        
        max_idx = np.argmax(similarities)
        if similarities[max_idx] > threshold:
            return {
                'question': self.questions[max_idx],
                'answer': self.answers[max_idx],
                'similarity': similarities[max_idx]
            }
        return None
    
    def generate_response(self, query: str) -> str:
        """Generate response using Groq's Llama-3-8b"""
        if any(greeting in query.lower() for greeting in ["hi", "hello", "how are you"]):
            return "I'm a medical AI assistant focused on cancer-related questions. How can I help you today?"
        
        medical_terms = ["diabetes", "heart disease", "alzheimer's"]
        if any(term in query.lower() for term in medical_terms):
            return "I specialize in cancer-related information. For other medical questions, please consult a healthcare professional."
        
        retrieved = self.retrieve_most_similar(query)
        if not retrieved:
            return "I couldn't find relevant information about that cancer topic. Please try rephrasing your question."
        
        chat_completion = self.client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {
                    "role": "system",
                    "content": "You are a medical assistant specializing in cancer information. Provide concise, accurate answers based on the given context. please try to act like a doctor as much as possible and give answers in a doctor like way dont just copy from the context i provide"
                },
                {
                    "role": "user",
                    "content": f"Context: {retrieved['answer']}\n\nQuestion: {query}\nAnswer:"
                }
            ],
            temperature=0.3,
            max_tokens=512
        )
        
        return chat_completion.choices[0].message.content

In [6]:
print("Initializing Medical RAG System with Groq...")
rag_system = MedicalRAGSystem()

test_queries = [
    "what is metastasis in general?",
    "hey i was just diagnosed with breast cancer what are the following steps i should do"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    print("Response:")
    print(rag_system.generate_response(query))
    print("\n" + "="*80)

Initializing Medical RAG System with Groq...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...


Query: what is metastasis in general?
Response:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Metastasis, in general, refers to the spread of cancer cells from the primary tumor site to other parts of the body through the bloodstream or lymphatic system. This process allows cancer cells to establish new tumors in distant organs, which can lead to the development of secondary cancers.

In the context of prostate cancer, metastasis typically occurs when cancer cells break away from the primary tumor in the prostate and travel to other parts of the body, such as the bones, lymph nodes, or other organs. Once established, these secondary tumors can produce symptoms, affect organ function, and impact overall quality of life.

In prostate cancer, metastasis is often referred to as "bone metastasis" or "bone mets," as the bones are a common site of spread. However, metastasis can also occur in other areas, such as the lymph nodes, liver, or lungs.

It's essential to note that metastasis is a critical aspect of cancer progression, and its detection is crucial for accurate staging and tr

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Congratulations on taking the first step towards addressing your breast cancer diagnosis. As your medical assistant, I'm here to guide you through the next steps.

Firstly, please take a deep breath and know that you're not alone in this journey. Breast cancer is a treatable condition, and with the right treatment and support, you can overcome it.

Here are the next steps you should take:

1. Schedule a meeting with your oncologist: Your oncologist will be your primary care provider throughout your treatment journey. They will discuss your diagnosis, treatment options, and create a personalized plan tailored to your needs.
2. Get a second opinion: It's essential to get a second opinion from another oncologist to confirm your diagnosis and treatment plan. This will give you peace of mind and ensure you're getting the best possible care.
3. Undergo further testing: Your oncologist may recommend additional tests to determine the extent of your cancer, such as a mammogram, ultrasound, MRI,